In [1]:
%load_ext google.cloud.bigquery
%matplotlib inline
from google.cloud import bigquery
client = bigquery.Client()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

In [2]:
def create_sql(database, table):
    return '''
            SELECT
                *
            FROM `{}.{}`
            LIMIT 15
            '''.format(database, table)




### transactions

In [95]:
database = 'bigquery-public-data.ethereum_blockchain'
table = 'transactions'
sql = create_sql(database, table)

df = client.query(sql).to_dataframe()
df.head()['hash'][0]

'0x1d023549fae00a06c5aa993846894c31f6278f8941890d606ac68634aac032f0'

### cell magic

In [59]:
dataset = client.dataset

In [93]:
%%bigquery txns
SELECT
    *
FROM `bigquery-public-data.ethereum_blockchain.transactions`
WHERE to_address IS NULL
LIMIT 15

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xe6b02f41e5b4438a987f908bb919eca274807c5d823a...,105,1,0x0e95adb39a2dfe1c8f3969de4480a5d3ddfadebd,None,0,800000,20000000000,0x606060405260008054600160a060020a031916331790...,313670,292670,0x54676cd2f1e8d296e7ac69f69675f936c4629dde,0x2c624ed478d2a0efb53b4494260746794465784820d2...,None,2016-12-11 03:41:34+00:00,2786327,0x90938f89f452d204b7477310e9797989d1e724d0e3c3...
1,0x4543c1eca44543e0232a296d0b7ebbbbfe990f99d4b9...,95,1,0x0e95adb39a2dfe1c8f3969de4480a5d3ddfadebd,None,0,800000,20000000000,0x606060405260008054600160a060020a031916331790...,263359,242359,0x04f0c372cdce16924ecbcc443c38ce996dce2202,0x74b5fa3b2dc03d17985d845c585769dbeb857057e9bb...,None,2016-12-11 00:16:59+00:00,2785477,0x97df6b6d30aa33919cf953a7e5a6081bba49a409bdba...
2,0xe25feed0854871e21b8b8f532ac823fe076d8b2c354a...,99,8,0x0e95adb39a2dfe1c8f3969de4480a5d3ddfadebd,None,0,800000,20000000000,0x606060405260008054600160a060020a031916331790...,513018,278360,0x5df775f6b6cb6ad2b7e863dc5c4bf66c18d63416,0xf0937882900b07958b5d61864e3d733f208b1056b436...,None,2016-12-11 03:04:55+00:00,2786163,0x4eea302adbfb7c8bc2acd098789ce78c5fc23d5f6637...
3,0x404ebdda8b5a98d61f2000fa3b92069c1a041486d00c...,101,6,0x0e95adb39a2dfe1c8f3969de4480a5d3ddfadebd,None,0,800000,20000000000,0x606060405260008054600160a060020a031916331790...,424706,298706,0xe0885628cba5d0d7db55e670a9f2e24a93eaa6ac,0x8029bc58aa2f7e6e602a71bc2cc0989637601c882bde...,None,2016-12-11 03:09:40+00:00,2786191,0xc4289912fe0d939f61bddbc36bf693b6f8877ef5508c...
4,0x362395b284d7e46ca6d5b5dcb7e671d4010c23d28fc4...,5,8,0x00ea6e2e39329489b7b2c3e9d1fd62f747f809bc,None,0,800000,20000000000,0x6060604052600261010860005055604051611c453803...,1695534,800000,0x6150cbd111be39e382f969c89c07ae4f200814a2,0xcede9ee0005dc2ec1e83373e1163d5161281e70702ea...,None,2016-12-11 05:59:00+00:00,2786909,0x011cee027cde26992ecd26fd01eba65f786c41de5122...
5,0x9c771607799845544b762f03237a10978613a4333c93...,9,16,0x00ea6e2e39329489b7b2c3e9d1fd62f747f809bc,None,0,1600000,20000000000,0x6060604052600261010860005055604051611c453803...,3315000,1600000,0x64dd210c3b1bf7f8a1dcefcbb06d81b603b2e8e5,0x54a2b2ae715d0d68af693555a03030d1aa67a4f73b85...,None,2016-12-11 05:59:41+00:00,2786912,0xb3de757ab834ca70507d3b159da8f8a32a43e4a7af9a...
6,0xe8cd214a11f7015d0c60414b76c680e3f22c86ea3e33...,92,8,0x0e95adb39a2dfe1c8f3969de4480a5d3ddfadebd,None,0,800000,20000000000,0x606060405260008054600160a060020a031916331790...,511586,264774,0xee939dbde9dec924bbe18c59ea8883d2e87d2332,0xd13d1ecc6a7f8ceefb32b411faf13d0ea1ae58687404...,None,2016-12-11 00:06:13+00:00,2785428,0x9ebe32354a6c5c1317b28403baab0712e28dbd7f79b9...
7,0xb8f228a45a680a72fd6baf6fc43ef52a0a024e8ac6c8...,5490,8,0xe9eeaec75883f0e389a78e2260bfac1776df2f1d,None,0,650000,20000000000,0x606060408181526001805460a060020a60ff02191690...,1970326,297169,0x70747726fd79cf73fb3055d5350f9248379081ad,0x15231e821cd1955560b33ed7d1edc2bf76ac9e7e5213...,None,2016-12-11 10:50:03+00:00,2788166,0x3c39cac96600f0c0bf0cdb9196d85c99a10c6cffe7dc...
8,0x2ec0ed35fe2d7d77a19a3b0882f15d36a919b96c8474...,5502,14,0xe9eeaec75883f0e389a78e2260bfac1776df2f1d,None,0,650000,20000000000,0x606060408181526001805460a060020a60ff02191690...,2448687,297169,0x885a665b654ed42e0a846169267e3eb547406b6d,0xbc7af7ee4011d762179114e94d2d09e48ab4fb5c20dd...,None,2016-12-11 10:50:32+00:00,2788169,0x83f57327e1adff751baa2bd3973580129513403dff0b...
9,0xcfc0502ba22b968664ebd404543d40a24ed538d07ab6...,5462,9,0xe9eeaec75883f0e389a78e2260bfac1776df2f1d,None,0,650000,20000000000,0x606060408181526001805460a060020a60ff02191690...,2064629,297169,0x6d628169dd66be184e5fc21abf3c36befee2e546,0x82b85bf241e36ec51c4aba866717c4ef3d72910008db...,None,2016-12-11 04:56:17+00:00,2786644,0xf3f3a998bb7288622fa1cdfb043aa68e417af21683d3...


In [61]:
dataset = client.dataset('bigquery-public-data.ethereum_blockchain')

In [65]:
tables = list(client.list_tables('bigquery-public-data.ethereum_blockchain'))

In [ ]:
database = 'bigquery-public-data.ethereum_blockchain'
table = 'blocks'
sql = create_sql(database, table)

df = client.query(sql).to_dataframe()
df.head()

subset transactions table to manageable number of rows: 20,000? Trying with most recent

In [113]:
%%bigquery
WITH in_out AS
    (WITH subset AS(SELECT *
                FROM `bigquery-public-data.ethereum_blockchain.transactions`
                ORDER BY block_timestamp DESC
                LIMIT 20000),

    f AS(SELECT DISTINCT from_address, count(*) as num_outgoing_txns
        FROM subset
        GROUP BY from_address),
    t AS(SELECT DISTINCT to_address, count(*) as num_incoming_txns
        FROM subset
        GROUP BY to_address)
    SELECT COALESCE(from_address,to_address) as addr,
            COALESCE(num_outgoing_txns,0) as outgoing,
            COALESCE(num_incoming_txns,0) as incoming
    FROM f FULL JOIN t
    ON f.from_address = t.to_address)

SELECT *, (outgoing + incoming) as total
FROM in_out
ORDER by total DESC
LIMIT 10


,addr,outgoing,incoming,total
0,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1386,0,1386
1,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,863,0,863
2,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0,514,514
3,0xd1ceeeeee83f8bcf3bedad437202b6154e9f5405,0,495,495
4,0x8fdcc30eda7e94f1c12ce0280df6cd531e8365c5,0,468,468
5,0xa7a7899d944fe658c4b0a1803bab2f490bd3849e,449,0,449
6,0x84295d5e054d8cff5a22428b195f5a1615bd644f,431,0,431
7,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,124,255,379
8,0x06012c8cf97bead5deae237070f9587f8e7a266d,0,354,354
9,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,271,0,271


need to default to zero when no transactions are present

In [ ]:
%%bigquery
WITH subset AS(SELECT *
            FROM `bigquery-public-data.ethereum_blockchain.transactions`
            WHERE MOD(ABS(FARM_FINGERPRINT(`hash`)), 15000)=0),
SELECT 